# Combining neural networks and reinforcement learning. 
1. Use deep neural networks to estimate the q values to each state/Action pair. Input - State and Output - q(s,a1),q(s,a2) etc. 
2. Stack of consequetive frames are used as input to the network. 

<img src="dqn1.png"/>


Experience replay: 
we store the agents experience in a dataset called replay memory. We use these replay memory samples to train the network. 
using the sequential data to train the network will lead to an ineffiecient network.  

<img src="dqn2.png"/>


1. Initialize replay memory capacity. 
2. Initialize the network with random weights. 
3. For each episode: 
    1. Initialie the starting state:
        - via exploration-exploitation strategy. 
    2. Execute selected action on emulator. 
    3. Observe reward and next state. 
    4. Store experience in replay memory. 
 